<a href="https://colab.research.google.com/github/AnikethDandu/traffic-sign-classification/blob/main/TrafficSignClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional Neural Network class

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
  def __init__(self):
    # Assume input image is 32x32x3
    self.PADDING_SIZE = 1
    self.KERNEL_SIZE = 3
    self.STRIDE = 1
    self.POOL_SIZE = 2
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv2 = nn.Conv2d(32, 64, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv3 = nn.Conv2d(64, 128, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv4 = nn.Conv2d(128, 256, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.fc1 = nn.Linear(2304, 512)
    self.fc2 = nn.Linear(512, 43)
    
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv2(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv3(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv4(x)), self.POOL_SIZE)
    x = x.flatten(start_dim=1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


    

In [19]:
import numpy as np
import cv2
import os

class DataProcessing():
  def __init__(self):
    self.IMG_SIZE = 50
    self.CLASS_COUNT = 43
    self.TRAIN_PATH = 'traffic_sign_images/Train'
    self.TEST_PATH = 'traffic_sign_images/Test'
    self.testing_data = []
    self.training_data = []
    self.training_folder_paths = []
    self.class_labels = {i: 0 for i in range(self.CLASS_COUNT)}

  def make_train_set(self):
    self.training_folder_paths = [self.TRAIN_PATH + f'/{str(i)}' for i in range(self.CLASS_COUNT)]
    for path in self.training_folder_paths:
      for image in os.listdir(path):
        image_class = int(self.training_folder_paths.index(path))
        image = cv2.imread(os.path.join(path, image), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (self.IMG_SIZE, self.IMG_SIZE))
        # TODO: Check possible need for shuffling data
        self.training_data.append([np.array(image), np.eye(self.CLASS_COUNT)[image_class]])
    np.random.shuffle(self.training_data)
    np.save('training_data.npy', self.training_data, allow_pickle=True)

  def make_test_set(self):
    class_labels = {}
    with open('traffic_sign_images/Test.csv', 'r') as file:
      for line in file.readlines():
        line_array = line.split(',')
        if line_array[0] != 'Width':
          image_index = line_array[7][-10:-5].lstrip('0')
          class_labels[image_index] = int(line_array[6])
    for image in os.listdir(self.TEST_PATH):
      if image[0] != 'G':
        image_path = class_labels[image[0:5].lstrip('0')]
        image = cv2.imread(os.path.join(self.TEST_PATH, image), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (self.IMG_SIZE, self.IMG_SIZE))
        self.testing_data.append([np.array(image), np.eye(self.CLASS_COUNT)[image_path]])
    np.random.shuffle(self.testing_data)
    np.save('testing_data.npy', self.testing_data, allow_pickle=True)


    

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import numpy as np

EPOCHS = 1
BATCH_SIZE = 32

created_npy = False

training_data = []
testing_data = []

DataProccesor = DataProcessing()
if 'training_data.npy' not in os.listdir():
  DataProccesor.make_train_set()
if 'testing_data.npy' not in os.listdir():
  DataProccesor.make_test_set()
training_data = np.load('training_data.npy', allow_pickle=True)
testing_data = np.load('testing_data.npy', allow_pickle=True)

total_images = 0
class_img_count = []
for folder in os.listdir(DataProccesor.TRAIN_PATH):
  if folder != '.DS_Store':
    image_count = len([img for img in os.listdir(os.path.join(DataProccesor.TRAIN_PATH, folder))])
    class_img_count.append(image_count)
    total_images += image_count

FINAL_WEIGHTS = torch.Tensor([1 - img_count/total_images for img_count in class_img_count])
CNN = ConvNet()
OPTIMIZER = optim.Adam(CNN.parameters(), lr=0.001)
CRITERION = nn.CrossEntropyLoss(weight=FINAL_WEIGHTS)

TRAIN_IMAGES = torch.tensor([i[0] for i in tqdm(training_data)]).view(-1, 3, 50, 50)
TRAIN_IMAGES = TRAIN_IMAGES / 255.0
TRAIN_LABELS = torch.tensor([i[1] for i in tqdm(training_data)], dtype=torch.float32)

TEST_IMAGES = torch.tensor([i[0] for i in tqdm(testing_data)]).view(-1, 3, 50, 50)
TEST_IMAGES = TEST_IMAGES / 255.0
TEST_LABELS = torch.tensor([i[1] for i in tqdm(testing_data)], dtype=torch.float32)

for epoch in range(EPOCHS):
  for i in tqdm(range(0, len(training_data), BATCH_SIZE)):
    BATCH_IMGS = TRAIN_IMAGES[i:i + BATCH_SIZE].view(-1, 3, 50, 50)
    BATCH_LABELS = TRAIN_LABELS[i:i + BATCH_SIZE]

    OPTIMIZER.zero_grad()
    OUTPUTS = CNN(BATCH_IMGS)
    LOSS = CRITERION(OUTPUTS, torch.tensor([list(label).index(1) for label in BATCH_LABELS]).long())
    LOSS.backward()
    OPTIMIZER.step()
  print(f'Loss: {LOSS}')

with torch.no_grad():
  class_correct = {}
  total_correct = 0
  total_images = 0
  total_classes = {}
  for i in tqdm(range(len(TEST_IMAGES))):
    CORRECT_CLASS = torch.argmax(TEST_LABELS[i])
    PREDICTED_CLASS = torch.argmax(CNN(TEST_IMAGES[i].view(-1, 3, 50, 50))[0])
    
    total_images += 1
    if PREDICTED_CLASS in total_classes:
      total_classes[PREDICTED_CLASS.item()] += 1
    else:
      total_classes[PREDICTED_CLASS.item()] = 1
    
    if PREDICTED_CLASS == CORRECT_CLASS:
      total_correct += 1
      if CORRECT_CLASS in class_correct:
        class_correct[CORRECT_CLASS.item()] += 1
      else:
        class_correct[CORRECT_CLASS.item()] = 1
print([f'Accuracy for {img_class}: {round(100 * class_correct[img_class] / total_classes[img_class])}' for img_class in class_correct])
print(f'Total accuracy: {round(100 * total_correct / total_images, 3)}%')




100%|██████████| 39209/39209 [00:00<00:00, 774858.72it/s]

100%|██████████| 39209/39209 [00:00<00:00, 1121974.03it/s]

100%|██████████| 12630/12630 [00:00<00:00, 960632.14it/s]

100%|██████████| 12630/12630 [00:00<00:00, 891924.29it/s]

100%|██████████| 1226/1226 [05:07<00:00,  3.99it/s]

  0%|          | 13/12630 [00:00<01:38, 128.48it/s]

Loss: 1.3850880861282349
Predicted: 11, Correct: 11
Predicted: 13, Correct: 13
Predicted: 11, Correct: 11
Predicted: 25, Correct: 25
Predicted: 39, Correct: 40
Predicted: 37, Correct: 37
Predicted: 14, Correct: 14
Predicted: 5, Correct: 4
Predicted: 1, Correct: 1
Predicted: 9, Correct: 9
Predicted: 10, Correct: 10
Predicted: 38, Correct: 2
Predicted: 10, Correct: 10
Predicted: 17, Correct: 17
Predicted: 13, Correct: 13
Predicted: 5, Correct: 5
Predicted: 28, Correct: 30
Predicted: 38, Correct: 38
Predicted: 12, Correct: 12
Predicted: 38, Correct: 38
Predicted: 1, Correct: 1
Predicted: 25, Correct: 25
Predicted: 13, Correct: 13
Predicted: 18, Correct: 22



  0%|          | 41/12630 [00:00<01:35, 132.17it/s]

Predicted: 2, Correct: 5
Predicted: 15, Correct: 15
Predicted: 1, Correct: 1
Predicted: 8, Correct: 17
Predicted: 13, Correct: 13
Predicted: 19, Correct: 19
Predicted: 4, Correct: 4
Predicted: 1, Correct: 1
Predicted: 26, Correct: 26
Predicted: 1, Correct: 1
Predicted: 42, Correct: 3
Predicted: 8, Correct: 8
Predicted: 26, Correct: 26
Predicted: 35, Correct: 35
Predicted: 2, Correct: 5
Predicted: 31, Correct: 31
Predicted: 4, Correct: 4
Predicted: 22, Correct: 22
Predicted: 1, Correct: 1
Predicted: 5, Correct: 5
Predicted: 26, Correct: 22
Predicted: 4, Correct: 4
Predicted: 4, Correct: 4
Predicted: 31, Correct: 31
Predicted: 33, Correct: 33
Predicted: 1, Correct: 1
Predicted: 1, Correct: 5
Predicted: 3, Correct: 3
Predicted: 35, Correct: 35
Predicted: 2, Correct: 2
Predicted: 37, Correct: 1



  1%|          | 72/12630 [00:00<01:29, 140.90it/s]

...

Predicted: 25, Correct: 25
Predicted: 36, Correct: 36
Predicted: 11, Correct: 25
Predicted: 30, Correct: 23
Predicted: 2, Correct: 4
Predicted: 8, Correct: 8
Predicted: 1, Correct: 1
Predicted: 3, Correct: 7
Predicted: 39, Correct: 1
Predicted: 20, Correct: 20
Predicted: 5, Correct: 7
Predicted: 23, Correct: 25
Predicted: 1, Correct: 3
Predicted: 26, Correct: 29
Predicted: 1, Correct: 8
Predicted: 2, Correct: 2
Predicted: 10, Correct: 10
Predicted: 23, Correct: 23
Predicted: 11, Correct: 27
Predicted: 26, Correct: 27
Predicted: 4, Correct: 30
Predicted: 10, Correct: 10



100%|█████████▉| 12620/12630 [01:48<00:00, 111.32it/s]

Predicted: 15, Correct: 15
Predicted: 2, Correct: 2
Predicted: 8, Correct: 8
Predicted: 13, Correct: 13
Predicted: 17, Correct: 17
Predicted: 2, Correct: 5
Predicted: 13, Correct: 13
Predicted: 11, Correct: 12
Predicted: 42, Correct: 42
Predicted: 30, Correct: 11
Predicted: 38, Correct: 38
Predicted: 38, Correct: 38
Predicted: 13, Correct: 13
Predicted: 9, Correct: 15
Predicted: 13, Correct: 13
Predicted: 18, Correct: 18
Predicted: 25, Correct: 18
Predicted: 34, Correct: 14
Predicted: 5, Correct: 3
Predicted: 12, Correct: 12
Predicted: 1, Correct: 3
Predicted: 35, Correct: 35
Predicted: 17, Correct: 17


100%|██████████| 12630/12630 [01:48<00:00, 115.95it/s]

Predicted: 9, Correct: 9
Predicted: 11, Correct: 11
Predicted: 38, Correct: 38
Predicted: 15, Correct: 15
Predicted: 13, Correct: 13
Predicted: 38, Correct: 38
Predicted: 3, Correct: 3
Predicted: 11, Correct: 11
Predicted: 30, Correct: 24
Predicted: 31, Correct: 5
['Accuracy for 11: 100', 'Accuracy for 13: 100', 'Accuracy for 25: 100', 'Accuracy for 37: 100', 'Accuracy for 14: 100', 'Accuracy for 1: 100', 'Accuracy for 9: 100', 'Accuracy for 10: 100', 'Accuracy for 17: 100', 'Accuracy for 5: 100', 'Accuracy for 38: 100', 'Accuracy for 12: 100', 'Accuracy for 15: 100', 'Accuracy for 19: 100', 'Accuracy for 4: 100', 'Accuracy for 26: 100', 'Accuracy for 8: 100', 'Accuracy for 35: 100', 'Accuracy for 31: 100', 'Accuracy for 22: 100', 'Accuracy for 33: 100', 'Accuracy for 3: 100', 'Accuracy for 2: 100', 'Accuracy for 39: 100', 'Accuracy for 18: 100', 'Accuracy for 7: 100', 'Accuracy for 34: 100', 'Accuracy for 20: 100', 'Accuracy for 16: 100', 'Accuracy for 42: 100', 'Accuracy for 28: 100'

In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/gdrive/My\ Drive/ColabNotebooks/Data/ traffic_sign_images.zip
!unzip traffic_sign_images.zip/traffic_sign_images.zip
!rm -r traffic_sign_images.zip/
'''